# Задание:

1. Найдите пример многозначного слова, для которого в топ-10 (метод most_similar) похожих на него слов входят слова связанные с разными значениями
2. Приведите пример трех слов w1, w2, w3, таких, что w1 и w2 являются синонимами, w1 и w3 являются антонимами, но при этом, similarity(w1, w2) < similarity(w1, w3)
3. Постройте решение на базе FastText для решаемой задачи на основе символьных n-грамм и сравните результат с предыдущими двумя. Какое количество символов брать - решите самостоятельно.

## Загрузка модулей:

In [1]:
!pip install gensim --quiet

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from gensim.utils import simple_preprocess
import numpy as np

print("Загрузка Word2Vec")
w2v_model = api.load("word2vec-google-news-300")

print("Загрузка FastText")
fasttext_model = api.load("fasttext-wiki-news-subwords-300")

print("Модели загружены.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 30.5 MB/s eta 0:00:00
Загрузка Word2Vec
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Загрузка FastText
[==================================================] 100.0% 958.5/958.4MB downloaded
Модели загружены.


## Задание 1:

In [2]:
word = "bank"

print("Word2Vec:")
for w, s in w2v_model.most_similar(word, topn=10):
    print(f"{w:15} {s:.4f}")

print("\nFastText:")
for w, s in fasttext_model.most_similar(word, topn=10):
    print(f"{w:15} {s:.4f}")

Word2Vec:
banks           0.7441
banking         0.6902
Bank            0.6699
lender          0.6342
banker          0.6093
depositors      0.6032
mortgage_lender 0.5798
depositor       0.5716
BofA            0.5715
Citibank        0.5590

FastText:
banks           0.8218
bank-           0.7700
banking         0.7487
bank.           0.7447
bank-to-bank    0.7348
bankʼs          0.7222
piggy-bank      0.7109
inter-bank      0.7089
bankcard        0.7068
bank-run        0.7040


## Задание 2:

In [3]:
w1 = "good"
w2 = "excellent"
w3 = "bad"

print("Word2Vec:")
print("sim(good, excellent) =", w2v_model.similarity(w1, w2))
print("sim(good, bad)       =", w2v_model.similarity(w1, w3))

print("\nFastText:")
print("sim(good, excellent) =", fasttext_model.similarity(w1, w2))
print("sim(good, bad)       =", fasttext_model.similarity(w1, w3))

Word2Vec:
sim(good, excellent) = 0.6442929
sim(good, bad)       = 0.7190051

FastText:
sim(good, excellent) = 0.7996695
sim(good, bad)       = 0.8503089


## Задание 3:

In [4]:
corpus = [
    "The bank approved the loan",
    "The river bank was flooded",
    "She deposited money in the bank",
    "The shoreline of the river was beautiful",
    "He is a good person",
    "She is an excellent student",
    "This is a bad example"
]

tokenized = [simple_preprocess(sentence) for sentence in corpus]

ft_ngram = FastText(
    sentences=tokenized,
    vector_size=100,
    window=3,
    min_count=1,
    min_n=3,
    max_n=5,
    epochs=200
)

## Проверки:

In [5]:
for w, s in ft_ngram.wv.most_similar("bank", topn=5):
    print(f"{w:15} {s:.4f}")

print("sim(good, excellent) =", ft_ngram.wv.similarity("good", "excellent"))
print("sim(good, bad)       =", ft_ngram.wv.similarity("good", "bad"))

deposited       0.2060
is              0.1870
good            0.1388
flooded         0.1324
this            0.1293
sim(good, excellent) = -0.07527339
sim(good, bad)       = 0.21121933
